# 以命令作業身分執行定型腳本

您可以使用適用于 Azure Machine Learning 的 Python SDK，將腳本提交為命令作業。 藉由使用作業，您可以輕鬆地追蹤定型機器學習模型時的輸入參數和輸出。

## 開始之前

您將需要最新版的 **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 使用 MLflow 進行自訂追蹤

以作業身分執行腳本時，您可以在定型腳本中使用 MLflow 來追蹤模型。 MLflow 可讓您追蹤您想要隨作業輸出一起儲存的任何自訂參數、計量或成品。

執行下列儲存格，以在**src**資料夾中建立**train-model-mlflow.py**腳本。 腳本會使用相同資料夾中 **diabetes.csv** 檔案來定型分類模型，該檔案會以引數的形式傳遞。 

檢閱下列程式碼，找出腳本將匯 `mlflow` 入和記錄：

- 正規化速率做為 **參數**。 
- 精確度和 AUC 作為 **計量**。
- 繪製的 ROC 曲線作為 **成品**。

In [ ]:
現在，您可以將腳本提交為命令作業。

執行下列儲存格來定型模型。 

In [ ]:
在 Studio 中，流覽至 **diabetes-train-mlflow** 作業，以探索您執行之命令作業的概觀：

- 在 [ **概觀** ] 索引標籤的 **[參數**] 底下尋找記錄的參數。
- 在 [計量] 索引標籤中尋找記錄 **的** 計量。
- 在 [ **映射** ] 索引標籤中尋找記錄的成品， (特別用於影像) ，然後在 [ **輸出 + 記錄** ] 索引標籤中， (所有檔案) 。

## 使用 MLflow 自動記錄

MLflow 也可以自動記錄任何參數、計量和成品，而不是使用自訂記錄。 使用 MLflow 自動記錄只需要一行程式碼。

執行下列儲存格，在**src**資料夾中建立**train-model-autolog.py**腳本。 腳本會使用相同資料夾中 **diabetes.csv** 檔案來定型分類模型，該檔案會以引數的形式傳遞。 

檢閱下列程式碼，找出腳本將匯 `mlflow` 入並啟用自動記錄程式程式碼： 

`mlflow.autolog()`

In [ ]:
現在，您可以將腳本提交為命令作業。

執行下列儲存格來定型模型。 

In [ ]:
在 Studio 中，流覽至 **diabetes-train-autolog** 作業，以探索您執行之命令作業的概觀：

- 在 [ **概觀** ] 索引標籤的 **[參數**] 底下尋找記錄的參數。
- 在 [計量] 索引標籤中尋找記錄 **的** 計量。
- 在 [ **映射** ] 索引標籤中尋找記錄的成品， (特別適用于影像) ，然後在 [ **輸出 + 記錄** ] 索引標籤中 (所有檔案，包括模型檔案) 。

## 使用 MLflow 來檢視和搜尋實驗

Azure Machine Learning Studio 是便於使用的 UI，可用來檢視和比較作業執行。 或者，您可以使用 MLflow 來檢視實驗作業。 

若要列出工作區中的作業，請使用下列命令來列出工作區中的實驗：


In [ ]:
若要擷取特定實驗，您可以依名稱取得它：

使用實驗名稱，您可以擷取該實驗的所有作業：

若要更輕鬆地比較作業執行和輸出，您可以設定搜尋來排序結果。 例如，下列儲存格會依 `start_time` 排序結果，而且只會顯示結果的最大值 `2` ： 

In [ ]:
您甚至可以建立查詢來篩選執行。 篩選查詢字串是以簡化版本的 SQL `WHERE` 子句撰寫。 

若要篩選，您可以使用兩種比較子類別：

- 數值比較子 (計量) ：=、！=、>、>=、<和<=。
- 字串比較子 (參數、標記和屬性) ： = 和 ！=。

深入瞭解 [如何使用 MLflow 追蹤實驗](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow)。

Now, you can submit the script as a command job.

Run the cell below to train the model. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In the Studio, navigate to the **diabetes-train-autolog** job to explore the overview of the command job you ran:

- Find the logged parameters in the **Overview** tab, under **Params**.
- Find the logged metrics in the **Metrics** tab.
- Find the logged artifacts in the **Images** tab (specifically for images), and in the **Outputs + logs** tab (all files, including the model files).

## Use MLflow to view and search for experiments

The Azure Machine Learning Studio is an easy-to-use UI to view and compare job runs. Alternatively, you can use MLflow to view experiment jobs. 

To list the jobs in the workspace, use the following command to list the experiments in the workspace:


In [ ]:
import mlflow
experiments = mlflow.list_experiments()
for exp in experiments:
    print(exp.name)

To retrieve a specific experiment, you can get it by its name:

In [ ]:
experiment_name = "diabetes-training"
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

Using an experiment name, you can retrieve all jobs of that experiment:

In [ ]:
mlflow.search_runs(exp.experiment_id)

To more easily compare job runs and outputs, you can configure the search to order the results. For example, the following cell orders the results by `start_time`, and only shows a maximum of `2` results: 

In [ ]:
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)

You can even create a query to filter the runs. Filter query strings are written with a simplified version of the SQL `WHERE` clause. 

To filter, you can use two classes of comparators:

- Numeric comparators (metrics): =, !=, >, >=, <, and <=.
- String comparators (params, tags, and attributes): = and !=.

Learn more about [how to track experiments with MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

In [ ]:
query = "metrics.AUC > 0.8 and tags.model_type = 'LogisticRegression'"
mlflow.search_runs(exp.experiment_id, filter_string=query)